In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Embedding, LSTM, Input, Activation, Permute, Dropout, add, dot, concatenate
from keras.utils import to_categorical
from pickle import dump, load
from nltk.corpus import words
import nltk
import spacy
import keras

import numpy as np
import random
import re

random.seed(42)

In [13]:
nlp = spacy.load('en_core_web_sm')

In [4]:
%reset -f

## Load Dataset

In [6]:
with open('qa6_yes-no-questions_test.txt', 'rb') as f:
    text_test = f.read().decode('utf-8')

with open('qa6_yes-no-questions_train.txt', 'rb') as f:
    text_train = f.read().decode('utf-8')

In [7]:
text_test;

In [8]:
text_train;

## Data Preprocessing

In [9]:
def preprocessing_data(txt):
    data = []
    record = []
    temp = None
    start = True
    for sentence in txt.split('\n'):
        doc = nlp(sentence)
        token = [i.text for i in doc]
        if len(token) == 0:
            break
        if start:
            if token[0] == '1':
                temp = []
            if temp:
                record.append(temp)
                record[0].extend(token[1:])
                temp = []
            else:
                record.append(token[1:])
            start = False
        elif '?' in token:
            record.append(token[1:-4])
            record.append(token[-3])
            temp = record[0].copy()
            data.append(record.copy())
            record = []
            start = True
        else:
            record[0].extend(token[1:])
    return data

In [14]:
train_data = preprocessing_data(text_train)
train_data;

In [15]:
test_data = preprocessing_data(text_test)
test_data;

## Create a Vocabulary to Store all Words

In [16]:
vocab = set()

In [17]:
merged_data = train_data + test_data

In [18]:
for story, question, _ in merged_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [19]:
vocab.add('yes')
vocab.add('no')
vocab;

In [20]:
VOCAB_LEN = len(vocab) + 1
VOCAB_LEN;

In [21]:
MAX_STORY_LEN = max(len(data[0]) for data in merged_data)
MAX_STORY_LEN;

In [22]:
MAX_QUESTION_LEN = max(len(data[1]) for data in merged_data)
MAX_QUESTION_LEN;

## Vectorize the Data

In [23]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [24]:
tokenizer.word_index;

In [25]:
# train_story_text = []
# train_question_text = []
# train_answers = []

# for story,question,answer in train_data:
#     train_story_text.append(story)
#     train_question_text.append(question)

# len(train_story_text)

In [26]:
# train_story_seq = tokenizer.texts_to_sequences(train_story_text)
# len(train_story_seq)

In [27]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=MAX_STORY_LEN, max_question_len=MAX_QUESTION_LEN):
    story_vectors = []
    question_vectors = []
    answer_indices = []  # Use indices instead of one-hot vectors

    for story, question, answer in data:
        # Vectorize the story and question
        story_vectors.append(
            [word_index.get(word.lower(), 0) for word in story[-max_story_len:]]
        )
        question_vectors.append(
            [word_index.get(word.lower(), 0) for word in question[-max_question_len:]]
        )

        # Store the answer as an index
        answer_indices.append(word_index.get(answer.lower(), 0))

    # Pad all sequences at once
    story_vectors = pad_sequences(story_vectors, maxlen=max_story_len)
    question_vectors = pad_sequences(question_vectors, maxlen=max_question_len)
    answer_indices = np.array(answer_indices)

    return story_vectors, question_vectors, answer_indices


In [28]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [29]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [30]:
inputs_train;

In [31]:
inputs_test;

In [32]:
answers_train = to_categorical(answers_train, num_classes=(len(tokenizer.word_index) + 1))
print(answers_train.shape)
answers_train;

(10000, 38)


In [33]:
answers_test = to_categorical(answers_test, num_classes=VOCAB_LEN)
print(answers_test.shape)
answers_test;

(1000, 38)


## Create End-to-End Memory Network (MemN2N) Model

### Input Placeholders for the Stories and Questions

In [34]:
input_sequence = Input((MAX_STORY_LEN, ))
question = Input((MAX_QUESTION_LEN, ))

### Encoders

In [35]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=VOCAB_LEN,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [36]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=VOCAB_LEN,output_dim=MAX_QUESTION_LEN))
input_encoder_c.add(Dropout(0.3))

In [37]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=VOCAB_LEN, output_dim=64, input_length=MAX_QUESTION_LEN))
question_encoder.add(Dropout(0.3))

### Encode the Sequences

In [38]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [39]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [40]:
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [41]:
answer = concatenate([response, question_encoded])
answer = LSTM(32)(answer)  # (samples, 32)
answer = Dropout(0.5)(answer)
answer = Dense(VOCAB_LEN)(answer)  # (samples, vocab_size)
answer = Activation('softmax')(answer)

In [42]:
model = Model([input_sequence, question], answer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary();

## Train the Model

In [43]:
history = model.fit([inputs_train, queries_train], answers_train,batch_size=128,epochs=4,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/4
79/79 [==============================] - 2617s 33s/step - loss: 2.0367 - accuracy: 0.4627 - val_loss: 1.2077 - val_accuracy: 0.4970
Epoch 2/4
79/79 [==============================] - 2614s 33s/step - loss: 1.1786 - accuracy: 0.5081 - val_loss: 0.8328 - val_accuracy: 0.4970
Epoch 3/4
79/79 [==============================] - 2604s 33s/step - loss: 0.9431 - accuracy: 0.5038 - val_loss: 0.7520 - val_accuracy: 0.4970
Epoch 4/4
79/79 [==============================] - 2608s 33s/step - loss: 0.8563 - accuracy: 0.4942 - val_loss: 0.7255 - val_accuracy: 0.4970


In [44]:
filename = 'memn2n_small_epochs.h5'
model.save(filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>